In [1]:
# Makes print and division act like Python 3
from __future__ import print_function, division

# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Enable inline plotting at lower left
%matplotlib inline
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'none'

# seaborn package for making pretty plots, but not necessary
try:
    import seaborn as sns
    params =   {'xtick.direction': 'in', 'ytick.direction': 'in', 'font.family': ['serif'],
                'text.usetex': True, 'text.latex.preamble': ['\usepackage{gensymb}']}
    sns.set_style("ticks", params)
except ImportError:
    print('Seaborn module is not installed.')
    
from IPython.display import display, Latex, clear_output

In [2]:
import pynrc
from pynrc import nrc_utils
from pynrc.nrc_utils import (webbpsf, poppy, pix_noise, S)

pynrc.setup_logging('WARNING', verbose=False)

from astropy.table import Table

In [3]:
# Initiate a NIRCam observation
nrc = pynrc.NIRCam('F444W', pupil='GRISM0', wind_mode='STRIPE', ypix=64)

# We want to optimize SNR of a K=16 M2V star
bp_k = S.ObsBandpass('johnson,k')
sp_M2V = pynrc.stellar_spectrum('M2V', 10, 'vegamag', bp_k)

nrc.update_detectors(read_mode='DEEP2', nint=35, ngroup=5)
snr1=nrc.sensitivity(sp=sp_M2V, forwardSNR=True, verbose=True)
nrc.update_detectors(read_mode='MEDIUM2', nint=35, ngroup=9)
snr2=nrc.sensitivity(sp=sp_M2V, forwardSNR=True, verbose=True)

#_ = nrc.sat_limits(verbose=True)
#_ = nrc.sensitivity(verbose=True, units='vegamag')

F444W SNR for M2V source:
Wave       SNR      Flux (uJy)
3.90   1111.09  29767.23
4.00   1169.43  28361.64
4.10   1121.30  27049.16
4.20   1071.35  26158.33
4.30    978.24  23226.08
4.40    922.63  22468.60
4.50    845.03  20824.46
4.60    741.44  17733.72
4.70    679.41  17170.92
4.80    608.38  15411.13
4.90    622.94  19485.87
5.00    344.29  17055.91
F444W SNR for M2V source:
Wave       SNR      Flux (uJy)
3.90   1114.49  29767.23
4.00   1172.67  28361.64
4.10   1124.67  27049.16
4.20   1074.86  26158.33
4.30    982.04  23226.08
4.40    926.63  22468.60
4.50    849.33  20824.46
4.60    746.21  17733.72
4.70    684.51  17170.92
4.80    613.90  15411.13
4.90    628.36  19485.87
5.00    351.72  17055.91


In [4]:
# Initiate a NIRCam observation
nrc = pynrc.NIRCam('F444W', pupil=None, wind_mode='STRIPE', ypix=64)
#print(nrc.sat_limits())

# We want to optimize SNR of a K=16 M2V star
bp_k = S.ObsBandpass('johnson,k')
sp_M2V = pynrc.stellar_spectrum('M2V', 15, 'vegamag', bp_k)

In [5]:
t = nrc.ramp_optimize(sp_M2V, tacq_max=1000, verbose=True, return_full_table=True, ng_min=5, snr_frac=0, \
                     well_frac_max=0.5)

BRIGHT1
BRIGHT2
DEEP2
DEEP8
MEDIUM2
MEDIUM8
RAPID
SHALLOW2
SHALLOW4
Top 10 results sorted by 'efficiency' (SNR/t_acq):
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
MEDIUM2      10   32     31.34   1002.76   1013.66   2501.7    0.465   78.574
MEDIUM2       9   35     27.93    977.55    989.47   2465.9    0.415   78.390
DEEP2         5   36     27.93   1005.48   1017.74   2497.9    0.415   78.298
MEDIUM2       8   41     24.52   1005.48   1019.45   2495.2    0.364   78.148
MEDIUM2       7   47     21.12    992.54   1008.55   2471.3    0.313   77.817
MEDIUM2       6   57     17.71   1009.57   1028.98   2481.1    0.263   77.347
SHALLOW2     10   63     16.01   1008.55   1030.00   2477.4    0.238   77.191
SHALLOW2      9   71     14.31   1015.70   1039.88   2477.1    0.212   76.815
MEDIUM2       5   71     14.31   1015.70   1039.88   2471.3    0.212   76.636
SHALLOW2      8   81   

In [12]:
print(t[t['Pattern']=='MEDIUM2'])
print(t[t['Pattern']=='DEEP2'])

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
MEDIUM2       5   71     14.31   1015.70   1039.88   2435.7    0.212   75.532
MEDIUM2       6   57     17.71   1009.57   1028.98   2422.1    0.263   75.507
MEDIUM2       7   49     21.12   1034.77   1051.46   2444.7    0.313   75.392
MEDIUM2       8   42     24.52   1030.00   1044.31   2431.8    0.364   75.250
MEDIUM2       9   37     27.93   1033.41   1046.01   2429.1    0.415   75.106
MEDIUM2      10   33     31.34   1034.09   1045.33   2423.9    0.465   74.968
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
DEEP2         5   35     27.93    977.55    989.47   2421.3    0.415   76.975
DEEP2         6   29     34.74   1007.52   1017.40   2444.3    0.516   76.631
DEEP2         7   25     41.55   1038.86   1047.38   2469.1    0

In [13]:
#print(np.unique(t['Pattern']))
print(pynrc.pynrc_core.table_filter(t,2))

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID        10  358      3.41   1219.38   1341.32   2424.7    0.051   66.204
BRIGHT1      10  172      6.47   1113.11   1171.70   2427.4    0.096   70.915
BRIGHT2      10  166      6.81   1130.83   1187.37   2421.5    0.101   70.272
BRIGHT2       9  186      6.13   1140.36   1203.72   2423.1    0.091   69.841
SHALLOW2     10   66     16.01   1056.57   1079.05   2422.9    0.238   73.759
SHALLOW2      9   74     14.31   1058.62   1083.82   2427.3    0.212   73.729
SHALLOW4     10   66     16.69   1101.53   1124.01   2436.4    0.248   72.670
SHALLOW4      9   73     14.99   1094.04   1118.90   2427.4    0.222   72.568
MEDIUM2       5   71     14.31   1015.70   1039.88   2435.7    0.212   75.532
MEDIUM2       6   57     17.71   1009.57   1028.98   2422.1    0.263   75.507
MEDIUM8      10   33     33.38   1101.53   1112.77   2439.7    0

In [18]:
t = nrc.ramp_optimize(sp_M2V, tacq_max=1000, verbose=True, return_full_table=True, ng_min=5, snr_frac=0, \
                     ideal_Poisson=True, well_frac_max=0.5)

BRIGHT1
BRIGHT2
DEEP2
DEEP8
MEDIUM2
MEDIUM8
RAPID
SHALLOW2
SHALLOW4
Top 10 results sorted by 'efficiency' (SNR/t_acq):
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
MEDIUM2      10   32     31.34   1002.76   1013.66   2501.7    0.465   78.574
MEDIUM2       9   35     27.93    977.55    989.47   2465.9    0.415   78.390
DEEP2         5   36     27.93   1005.48   1017.74   2497.9    0.415   78.298
MEDIUM2       8   41     24.52   1005.48   1019.45   2495.2    0.364   78.148
MEDIUM2       7   47     21.12    992.54   1008.55   2471.3    0.313   77.817
MEDIUM2       6   57     17.71   1009.57   1028.98   2481.1    0.263   77.347
SHALLOW2     10   63     16.01   1008.55   1030.00   2477.4    0.238   77.191
SHALLOW2      9   71     14.31   1015.70   1039.88   2477.1    0.212   76.815
MEDIUM2       5   71     14.31   1015.70   1039.88   2471.3    0.212   76.636
SHALLOW2      8   81   

In [17]:
print(t[t['Pattern']=='MEDIUM2'])
print(t[t['Pattern']=='DEEP2'])

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
MEDIUM2      10   32     31.34   1002.76   1013.66   2501.7    0.465   78.574
MEDIUM2       9   35     27.93    977.55    989.47   2465.9    0.415   78.390
MEDIUM2       8   41     24.52   1005.48   1019.45   2495.2    0.364   78.148
MEDIUM2       7   47     21.12    992.54   1008.55   2471.3    0.313   77.817
MEDIUM2       6   57     17.71   1009.57   1028.98   2481.1    0.263   77.347
MEDIUM2       5   71     14.31   1015.70   1039.88   2471.3    0.212   76.636
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
DEEP2         8   21     48.37   1015.70   1022.85   2526.7    0.718   79.003
DEEP2         7   24     41.55    997.31   1005.48   2500.6    0.617   78.860
DEEP2         6   29     34.74   1007.52   1017.40   2508.5    0

In [6]:
t = nrc.ramp_optimize(sp_M2V, tacq_max=1400, tacq_frac=0.1, verbose=True)

BRIGHT1
BRIGHT2
DEEP2
DEEP8
MEDIUM2
MEDIUM8
RAPID
SHALLOW2
SHALLOW4
Top 10 results sorted by 'efficiency' (SNR/t_acq):
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
DEEP2         6   39     34.74   1354.95   1368.23    984.5    0.139    0.720
DEEP2         8   28     48.37   1354.27   1363.80    979.2    0.193    0.718
DEEP2         5   49     27.93   1368.57   1385.26    990.8    0.112    0.715
DEEP2         7   33     41.55   1371.30   1382.54    988.0    0.166    0.715
DEEP2         6   40     34.74   1389.69   1403.31    997.1    0.139    0.710
DEEP2         5   50     27.93   1396.50   1413.53   1000.8    0.112    0.708
DEEP2         7   34     41.55   1412.85   1424.43   1002.9    0.166    0.704
DEEP2         6   41     34.74   1424.43   1438.40   1009.4    0.139    0.702
DEEP2         5   51     27.93   1424.43   1441.80   1010.8    0.112    0.701
DEEP2         4   67   

In [8]:
t = nrc.ramp_optimize(sp_M2V, tacq_max=1000, tacq_frac=0.1, verbose=True, patterns=['MEDIUM2'], ng_min=9)

MEDIUM2
Top 10 results sorted by 'efficiency' (SNR/t_acq):
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
MEDIUM2       9   34     27.93    949.62    961.20    808.9    0.112    0.842
MEDIUM2      10   31     31.34    971.42    981.98    817.7    0.125    0.833
MEDIUM2       9   35     27.93    977.55    989.47    820.8    0.112    0.829
MEDIUM2      10   32     31.34   1002.76   1013.66    830.8    0.125    0.820
MEDIUM2       9   36     27.93   1005.48   1017.74    832.4    0.112    0.818
MEDIUM2      10   33     31.34   1034.09   1045.33    843.6    0.125    0.807


In [10]:
t['eff'] = t['SNR']/np.sqrt(t['t_acq'])
ind_sort = np.argsort(1 / t['eff'])
print(t[ind_sort])

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
DEEP2         5   35     27.93    977.55    989.47    837.4    0.112   26.620
DEEP2         5   37     27.93   1033.41   1046.01    860.9    0.112   26.620
DEEP2         6   28     34.74    972.78    982.32    834.2    0.139   26.616
DEEP2         6   29     34.74   1007.52   1017.40    849.0    0.139   26.616
DEEP2         6   30     34.74   1042.27   1052.48    863.5    0.139   26.616
DEEP2         7   23     41.55    955.75    963.59    824.8    0.166   26.572
DEEP2         7   24     41.55    997.31   1005.48    842.6    0.166   26.572
DEEP2         8   20     48.37    967.33    974.14    827.6    0.193   26.515
DEEP2         8   22     48.37   1064.07   1071.56    868.0    0.193   26.515
DEEP2         4   48     21.12   1013.66   1030.00    850.9    0.084   26.512
DEEP8         7   25     43.60   1089.95   1098.47    870.1    0

In [21]:
nrc.update_detectors(read_mode='DEEP2', nint=33, ngroup=7, verbose=True)

New Ramp Settings:
  read_mode :    DEEP2
  nf        :        2
  nd2       :       18
  ngroup    :        7
  nint      :       33
New Detector Settings
  wind_mode :   STRIPE
  xpix      :     2048
  ypix      :       64
  x0        :        0
  y0        :        0
New Ramp Times
  t_group   :    6.812
  t_frame   :    0.341
  t_int     :   41.554
  t_exp     : 1371.296
  t_acq     : 1382.536


{u't_acq': 0.68122,
 u't_exp': 0.34061,
 u't_frame': 0.34061,
 u't_group': 0.34061,
 u't_int': 0.34061}

In [16]:
out = nrc.sensitivity(sp=sp_M2V, forwardSNR=True, units='mJy', verbose=True)

F444W SNR for M2V source:
Wave       SNR  Flux (mJy)
3.90   1284.21     29.77
4.00   1350.93     28.36
4.10   1295.88     27.05
4.20   1238.78     26.16
4.30   1132.39     23.23
4.40   1068.90     22.47
4.50    980.37     20.82
4.60    862.34     17.73
4.70    791.76     17.17
4.80    711.05     15.41
4.90    727.58     19.49
5.00    411.91     17.06


In [3]:
nrc = pynrc.NIRCam(filter='F430M', ngroup=2, wind_mode='WINDOW', xpix=320, ypix=320)
nrc.sensitivity(units='vegamag')

5.11857712754 958.279930576 270.95500382


({u'Spectrum': u'Flat spectrum in photlam',
  u'nsig': 10,
  u'sensitivity': 15.795335211450428,
  u'units': 'vegamag'},
 {u'Spectrum': u'Flat spectrum in photlam',
  u'nsig': 10,
  u'sensitivity': 13.455851256881445,
  u'units': u'vegamag/arcsec^2'})

In [264]:
# Build ramp optimizer

# Input source spectrum
#  - define whether point source or extended 
# Input maximum desired well fill fraction
# Input minimum number of desired ramps
# Specify zodi bg level

nrc.sensitivity(units='vegamag')

({u'Spectrum': u'Flat spectrum in photlam',
  u'nsig': 10,
  u'sensitivity': 15.795335211450428,
  u'units': 'vegamag'},
 {u'Spectrum': u'Flat spectrum in photlam',
  u'nsig': 10,
  u'sensitivity': 13.455851256881445,
  u'units': u'vegamag/arcsec^2'})

In [46]:
def ramp_optimize_tacq(self, sp, sp_bright=None, is_extended=False, patterns=None,
    tacq_max=1000, well_frac_max=0.8, nint_min=1, nint_max=1000, ng_min=2, ng_max=None,
    tacq_frac=0.05, return_full_table=None, verbose=False):
    """
    Find the optimal ramp settings to observe spectrum based input constraints.
    This function quickly runs through each detector readout pattern and 
    calculates the acquisition time and SNR for all possible settings of NINT
    and NGROUP that fulfill the SNR requirement (and other constraints). 

    The final output table is then filtered, removing those exposure settings
    that have the same exact acquisition times but worse SNR. Further "obvious"
    comparisons are done that exclude settings where there is another setting
    that has both better SNR and less acquisition time. The best results are
    then sorted by an efficiency metric (SNR / acq_time). To skip filtering
    of results, set return_full_table=True.

    The result is an AstroPy Table.

    Parameters
    ==========
    sp          : A pysynphot spectral object to calculate SNR.
    sp_bright   : Same as sp, but optionaly used to calculate the saturation limit
                  (treated as brightest source in field). If a coronagraphic mask 
                  observation, then this source is assumed to be occulted and 
                  sp is unocculted.
    is_extended : Treat source(s) as extended objects, then in units/arcsec^2

    patterns      : List of a subset of MULTIACCUM patterns to check, otherwise check all.
    well_frac_max : Maximum level that the pixel well is allowed to be filled. 
                    Fractions greater than 1 imply hard saturation, but the reported 
                    SNR will not be aware of any saturation that may occur to sp.
    snr_min       : Minimum required SNR for source. For grism, this is the average
                    SNR for all wavelength.
    snr_frac      : Give fractional buffer room rather than strict SNR cut-off.
    nint_min/max  : Min/max number of desired integrations.
    ng_min/max    : Min/max number of desired groups in a ramp.

    return_full_table : Don't filter or sort the final results.
    verbose           : Print out top 10 results.

    Example
    ----------
    # What are the optimal exposure settings to obtain SNR~1000 spectral data
    # for an M2V star (K=10 mags) in the F444W filter?
    #
    # Initiate a NIRCam observation
    nrc = pynrc.NIRCam('F430M', pupil='GRISM0', wind_mode='STRIPE', ypix=64)

    # Set up M2V stellar spectrum normalized to K=10 mags
    bp_k = S.ObsBandpass('johnson,k')
    sp_M2V = pynrc.stellar_spectrum('M2V', 10, 'vegamag', bp_k)

    # Find optimal settings with the only constraint on the SNR
    out = ramp_optimize(sp_M2V, snr_min=1000, verbose=True)

    # In this case, the best case is DEEP2, ngroup=7, nint=33
    # Update nrc settings and print out sensitivities to verify
    nrc.update_detectors(read_mode='DEEP2', ngroup=7, nint=33)
    _ = nrc.sensitivity(sp=sp_M2V, forwardSNR=True, units='mJy', verbose=True)

    # Which should print out:		
    F444W SNR for M2V source:
    Wave       SNR      Flux (mJy)
    3.90    1284.2     29.77
    4.00    1350.9     28.36
    4.10    1295.9     27.05
    4.20    1238.8     26.16
    4.30    1132.4     23.23
    4.40    1068.9     22.47
    4.50     980.4     20.82
    4.60     862.3     17.73
    4.70     791.8     17.17
    4.80     711.1     15.41
    4.90     727.6     19.49
    5.00     411.9     17.06
    """

    def parse_snr(snr, grism_obs, ind_snr):
        if grism_obs:
            res = snr['snr']
            return np.mean(res)
        else:
            return snr[ind_snr]['snr']            


    pupil = self.pupil
    grism_obs = (pupil is not None) and ('GRISM' in pupil)
    dhs_obs   = (pupil is not None) and ('DHS'   in pupil)
    coron_obs = (pupil is not None) and ('LYOT'   in pupil)

    det_params_orig = self.det_info.copy()

    if dhs_obs:
        raise NotImplementedError('DHS has yet to be fully included.')
    if grism_obs and is_extended:
        raise NotImplementedError('Extended objects not implemented for grism observations.')

    # Brightest source in field
    if sp_bright is None:
        sp_bright = sp

    # Generate PSFs for faint and bright objects and get max pixel flux
    # Only necessary for point sources
    if is_extended:
        ind_snr = 1
        obs = S.Observation(sp, self.bandpass, binset=self.bandpass.wave)
        pix_count_rate = obs.countrate() * self.pix_scale**2
    else:
        ind_snr = 0

        if grism_obs:
            _, psf_bright = self.gen_psf(sp_bright, use_bg_psf=False)
            _, psf_faint  = self.gen_psf(sp, use_bg_psf=True)
        else:
            psf_bright = self.gen_psf(sp_bright, use_bg_psf=False)
            psf_faint  = self.gen_psf(sp, use_bg_psf=True)
        pix_count_rate = np.max([psf_bright.max(), psf_faint.max()])

    image = self.sensitivity(sp=sp, forwardSNR=True, return_image=True)

    # Cycle through each readout pattern
    if patterns is None:
        pattern_settings = self.multiaccum._pattern_settings
        patterns = pattern_settings.keys()
    patterns.sort()

    rows = []
    for read_mode in patterns:
        if verbose: print(read_mode)
        # Maximum allowed groups for given readout pattern
        if ng_max is None:
            _,_,ng_max = pattern_settings.get(read_mode)
        for ng in range(ng_min,ng_max+1):
            self.update_detectors(read_mode=read_mode, ngroup=ng, nint=1)

            # Get saturation level of observation
            mtimes = self.multiaccum_times
            well_frac = pix_count_rate * mtimes['t_int'] / self.well_level
            # If above well_frac_max, then this setting is invalid
            if well_frac > well_frac_max:
                continue

            # Approximate integrations needed to get to required t_acq
            t_acq = mtimes['t_acq']
            nint = int(tacq_max / t_acq)
            nint = np.max([nint_min,nint])
            if nint>nint_max:
                continue

            # Find NINT with t_acq > 0.95 tacq_max
            self.update_detectors(nint=nint)
            t_acq = mtimes['t_acq']
            while (t_acq<((1-tacq_frac)*tacq_max)) and (nint<=nint_max):
                nint += 1
                self.update_detectors(nint=nint)
                t_acq = mtimes['t_acq']
            if nint>nint_max:
                continue
                
            # Get SNR (assumes no saturation)
            snr = parse_snr(self.sensitivity(sp=sp, forwardSNR=True, image=image), \
                grism_obs, ind_snr)

            # Approximate integrations needed to get to required SNR
            nint = int((snr_min / snr)**2)
            nint = np.max([nint_min,nint])
            if nint>nint_max:
                continue

            # Find NINT with SNR > 0.95 snr_min
            self.update_detectors(nint=nint)
            snr = parse_snr(self.sensitivity(sp=sp, forwardSNR=True, image=image), \
                grism_obs, ind_snr)
            while (snr<((1-snr_frac)*snr_min)) and (nint<=nint_max):
                nint += 1
                self.update_detectors(nint=nint)
                snr = parse_snr(self.sensitivity(sp=sp, forwardSNR=True, image=image), \
                    grism_obs, ind_snr)
            if nint>nint_max:
                continue

            mtimes = self.multiaccum_times
            rows.append((read_mode, ng, nint, mtimes['t_int'], mtimes['t_exp'], \
                mtimes['t_acq'], snr, well_frac))

            # Increment NINT until SNR > 1.05 snr_min
            # Add each NINT to table output
            while snr < ((1+snr_frac)*snr_min):
                nint += 1
                self.update_detectors(nint=nint)
                snr = parse_snr(self.sensitivity(sp=sp, forwardSNR=True, image=image), \
                    grism_obs, ind_snr)
                if nint > nint_max:
                    continue
                mtimes = self.multiaccum_times
                rows.append((read_mode, ng, nint, mtimes['t_int'], mtimes['t_exp'], \
                    mtimes['t_acq'], snr, well_frac))

    # Return to original values
    self.update_detectors(**det_params_orig)

    if len(rows)==0:
        _log.warning('No ramp settings allowed within constraints! Reduce constraints.')
        return

    # Place rows into a
    t_all = Table(rows=rows, \
        names=('Pattern', 'NGRP', 'NINT', 't_int', 't_exp', 't_acq', 'SNR', 'Well'))
    t_all['Pattern'].format = '<10'
    t_all['t_int'].format = '9.2f'
    t_all['t_exp'].format = '9.2f'
    t_all['t_acq'].format = '9.2f'
    t_all['SNR'].format = '8.1f'
    t_all['Well'].format = '8.3f'

    t_all['eff'] = t_all['SNR'] / t_all['t_acq']
    t_all['eff'].format = '8.3f'

    # Sort by efficiency
    ind_sort = np.argsort(1 / t_all['eff'])
    t_all = t_all[ind_sort]

    if verbose: print("Top 10 results sorted by 'efficiency' (SNR/t_acq):")
    if return_full_table:
        if verbose: print(t_all[0:10])
        return t_all


    # Make a copy for filtering
    t = t_all.copy()

    # For equivalent acquisition times, remove worse SNR
    t_uniq = np.unique(t['t_acq'])
    ind_good = []
    for tacq in t_uniq:
        ind = np.where(t['t_acq']==tacq)[0]
        ind_snr_best = t['SNR'][ind]==t['SNR'][ind].max()
        ind_good.append(ind[ind_snr_best][0])
    t = t[ind_good]

    # For each remaining row, exlude those that take longer with worse SNR than any other row
    ind_bad = []
    ind_bad_comp = []
    for i,row in enumerate(t):
        for j,row_compare in enumerate(t):
            if i==j: continue
            if (row['t_acq']>row_compare['t_acq']) and (row['SNR']<=(1.001*row_compare['SNR'])):
                ind_bad.append(i)
                ind_bad_comp.append(j)
                break
    t.remove_rows(ind_bad)

    #ind_sort = np.lexsort((t['NGRP'],t['Pattern']))
    #t = t[ind_sort]

    if verbose: print(t[0:10])
    return t



In [7]:
nrc = pynrc.NIRCam(filter='F356W', pupil='GRISM0', ngroup=10, wind_mode='WINDOW', xpix=320, ypix=320)
snr = nrc.sensitivity(units='vegamag', verbose=True)

sp = pynrc.stellar_spectrum('flat', 14, 'vegamag', nrc.bandpass)
out = nrc.sensitivity(sp=sp, forwardSNR=True, verbose=True)
res = out['snr']
print(res[int(len(res) // 2)])

nrc.update_detectors(ngroup=2)
_ = nrc.sat_limits(verbose=True)

F356W Background Sensitivity (10-sigma) for Flat spectrum in photlam source:
Wave   Limit (vegamag)
3.10     10.72
3.20     14.05
3.30     14.03
3.40     14.08
3.50     14.03
3.60     14.00
3.70     13.94
3.80     13.85
3.90     13.70
4.00     11.83
F356W SNR for Flat spectrum in photlam source:
Wave    SNR     Flux (uJy)
3.10     0.39  604.59
3.20     8.01  624.09
3.30     9.12  643.59
3.40     9.58  663.09
3.50     9.84  682.60
3.60    10.33  702.10
3.70    10.52  721.60
3.80    10.46  741.11
3.90    10.04  760.61
4.00     2.16  780.11
10.3263939172
F356W Saturation Limit assuming G2V source:
Wave    Sat Lim (vegamag)
3.10     3.73
3.20     5.33
3.30     5.37
3.40     5.33
3.50     5.25
3.60     5.21
3.70     5.14
3.80     5.04
3.90     4.92
4.00     3.96


In [51]:
sp = pynrc.stellar_spectrum('flat', 6, 'vegamag', nrc.bandpass)
sp_bright = pynrc.stellar_spectrum('flat', 14, 'vegamag', nrc.bandpass)
rows = ramp_optimize(nrc, sp, snr_min=1000, well_frac_max=0.8, nint_min=5)

14872.9173186
RAPID
BRIGHT2
BRIGHT1
MEDIUM8
DEEP8
DEEP2
SHALLOW4
MEDIUM2
SHALLOW2


In [52]:
t_all = Table(rows=rows, names=('Pattern', 'NGRP', 'NINT', 't_int', 't_exp', 't_acq', 'SNR', 'Well'))
t_all['Pattern'].format = '<10'
t_all['t_int'].format = '9.2f'
t_all['t_exp'].format = '9.2f'
t_all['t_acq'].format = '9.2f'
t_all['SNR'].format = '8.1f'
t_all['Well'].format = '8.3f'

t = t_all.copy()

# For equivalent acquisition times, remove worse SNR
t_uniq = np.unique(t['t_acq'])
ind_good = []
for tacq in t_uniq:
    ind = np.where(t['t_acq']==tacq)[0]
    ind_snr_best = t['SNR'][ind]==t['SNR'][ind].max()
    ind_good.append(ind[ind_snr_best][0])
t = t[ind_good]

# For each remaining row, exlude those that take longer with worse SNR than any other row
ind_bad = []
for i,row in enumerate(t):
    for j,row_compare in enumerate(t):
        if i==j: continue
        if (row['t_acq']>row_compare['t_acq']) and (row['SNR']<=(1.01*row_compare['SNR'])):
            ind_bad.append(i)
            break
t.remove_rows(ind_bad)

ind_sort = np.lexsort((t['NGRP'],t['Pattern']))
t = t[ind_sort]
print(t)

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well  
---------- ---- ---- --------- --------- --------- -------- --------
RAPID         4    5      4.28     21.38     26.73   1019.8    0.795
RAPID         4    6      4.28     25.66     32.07   1117.1    0.795


In [49]:
# Sort
print('Sort by efficiency')
eff = t['t_acq'] / t['SNR']
ind_sort = np.argsort(eff)
print(t[ind_sort][0:5])

print('\nSort by min acq time')
ind_sort = np.argsort(t['t_acq'])
print(t[ind_sort][0:5])

Sort by efficiency
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well  
---------- ---- ---- --------- --------- --------- -------- --------
RAPID         4  480      4.28   2052.56   2565.70   9992.1    0.795

Sort by min acq time
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well  
---------- ---- ---- --------- --------- --------- -------- --------
RAPID         4  480      4.28   2052.56   2565.70   9992.1    0.795


In [50]:
print(t_all)
#print(t_all[t_all['Pattern']=='BRIGHT2'])

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well  
---------- ---- ---- --------- --------- --------- -------- --------
RAPID         3  710      3.21   2277.06   3036.07   9994.2    0.596
RAPID         3  711      3.21   2280.26   3040.35  10001.2    0.596
RAPID         3  712      3.21   2283.47   3044.63  10008.3    0.596
RAPID         3  713      3.21   2286.68   3048.90  10015.3    0.596
RAPID         3  714      3.21   2289.88   3053.18  10022.3    0.596
RAPID         3  715      3.21   2293.09   3057.45  10029.3    0.596
RAPID         3  716      3.21   2296.30   3061.73  10036.3    0.596
RAPID         3  717      3.21   2299.51   3066.01  10043.3    0.596
RAPID         3  718      3.21   2302.71   3070.28  10050.3    0.596
RAPID         3  719      3.21   2305.92   3074.56  10057.3    0.596
...         ...  ...       ...       ...       ...      ...      ...
BRIGHT1       2  730      3.21   2341.20   3121.60  10135.5    0.596
BRIGHT1       2  731      3.21   2

In [34]:
t['t_int'].format

In [26]:
print(nrc.multiaccum_times)
print(nrc.det_info)
nrc.sensitivity(sp=sp, forwardSNR=True)[0]['snr']

{u't_frame': 10.73677, u't_int': 10.73677, u't_exp': 10.73677, u't_acq': 21.47354, u't_group': 10.73677}
{u'nint': 1, u'nd3': 0, u'nf': 1, u'read_mode': u'RAPID', u'ypix': 2048, u'ngroup': 1, u'nd1': 0, u'y0': 0, u'x0': 0, u'nd2': 0, u'wind_mode': u'FULL', u'xpix': 2048}


2834.405588234561

In [12]:
# Input maximum acq time
# Output ramp settings that maximize SNR 
nrc.multiaccum.patterns_list

[u'CUSTOM',
 u'BRIGHT1',
 u'BRIGHT2',
 u'DEEP2',
 u'DEEP8',
 u'MEDIUM2',
 u'MEDIUM8',
 u'RAPID',
 u'SHALLOW2',
 u'SHALLOW4']

In [ ]:
nrc.update_detectors()

In [8]:
nrc.multiaccum._pattern_settings.keys()

[u'RAPID',
 u'BRIGHT2',
 u'BRIGHT1',
 u'MEDIUM8',
 u'DEEP8',
 u'DEEP2',
 u'SHALLOW4',
 u'MEDIUM2',
 u'SHALLOW2']